In [1]:
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

2024-06-03 10:37:47.359246: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-03 10:37:47.417901: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-03 10:37:47.418710: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 10:37:48.537676: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('spam_and_ham_classification.csv')
df.head(5)

,label,text
0,ham,into the kingdom of god and those that are ent...
1,spam,there was flow at hpl meter 1505 on april firs...
2,ham,take a look at this one campaign for bvyhprice...
3,spam,somu wrote actually thats what i was looking f...
4,spam,fathi boudra wrote i fixed the issue in the sv...


In [3]:
df.rename(columns={'text': 'email'}, inplace=True)

df['label'] = df['label'].map({'ham': 0, 'spam': 1})
df.head(5)

,label,email
0,0,into the kingdom of god and those that are ent...
1,1,there was flow at hpl meter 1505 on april firs...
2,0,take a look at this one campaign for bvyhprice...
3,1,somu wrote actually thats what i was looking f...
4,1,fathi boudra wrote i fixed the issue in the sv...


In [5]:
test_data = df['email'][:5]

test_data

0    into the kingdom of god and those that are ent...
1    there was flow at hpl meter 1505 on april firs...
2    take a look at this one campaign for bvyhprice...
3    somu wrote actually thats what i was looking f...
4    fathi boudra wrote i fixed the issue in the sv...
Name: email, dtype: object

In [6]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8081/v1/models/email-spam-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1717375942'}]}


In [7]:
def prepare_json(sms):
    feature_spec = {
        "email": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(sms, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

endpoint = "http://localhost:8081/v1/models/email-spam-detection-model:predict"

for text in test_data:
    json_data = prepare_json(text)
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()
    if prediction:
        prediction_value = prediction['predictions'][0][0]
        result = "Spam" if prediction_value > 0.5 else "Not Spam"
    else:
        result = "Error: No predictions found in response."

    print(result)


Not Spam
Spam
Not Spam
Spam
Spam
